In [26]:
import skmob, pymysql, csv, os
import pandas as pd
from datetime import datetime
from skmob.preprocessing import detection, compression

conn = pymysql.connect(
    host='127.0.0.1', user='root', password='root', db='USERS', charset='utf8')

with conn:
    with conn.cursor() as cur:
        sql = "SELECT distinct deviceid from tc_positions"
        cur.execute(sql)
        device_id_list = cur.fetchall()

        for device_id in device_id_list:
            device_id = str(device_id[0])
            sql = "SELECT * FROM tc_positions WHERE deviceid=" + device_id
            cur.execute(sql)
            trajectory_data = cur.fetchall()

            data_list = list()
            for data in trajectory_data:
                data_list.append(list([data[1], data[3], data[4], data[2]]))

            tdf = skmob.TrajDataFrame(
                data_list, user_id=0, latitude=1, longitude=2, datetime=3)
            stdf = detection.stay_locations(
                tdf, stop_radius_factor=0.5, minutes_for_a_stop=30.0, spatial_radius_km=0.2, leaving_time=True)
            
            print('Device id:\t\t\t\t%s' % device_id)
            print('Points of the original trajectory:\t%s' % len(tdf))
            print('Points of stops:\t\t\t%s' % len(stdf))

            stay_point_list = stdf.values.tolist()
            for stay_point in stay_point_list:
                stay_point[3] = stay_point[3].strftime('%Y-%m-%d %H:%M:%S')
                stay_point[4] = stay_point[4].strftime('%Y-%m-%d %H:%M:%S')
            fields = ['deviceid', 'latitude', 'longitude', 'arrive_time', 'leaving_time']
            path = "./staypoint/"+device_id
            if not os.path.isdir(path):
                os.mkdir(path)
            with open("./staypoint/"+device_id+'/stay_points.csv', 'w',newline='') as f:
                write = csv.writer(f)
                write.writerow(fields)
                write.writerows(stay_point_list)
            
            stay_point_dict = dict()
            for idx in range(len(stay_point_list)):
                date_time = stay_point_list[idx][3][0:10]
                if date_time not in stay_point_dict.keys():
                    stay_point_dict[date_time] = list()
                stay_point_dict[date_time].append(stay_point_list[idx])
            
            for key in stay_point_dict.keys():
                with open("./staypoint/"+device_id+'/'+key+'.csv', 'w', newline='') as f:
                    write = csv.writer(f)
                    write.writerow(fields)
                    write.writerows(stay_point_dict[key])        

# tdf.plot_trajectory(zoom=12, weight=3, opacity=0.9, tiles='Stamen Toner')
# m = stdf.plot_trajectory(max_users=1, start_end_markers=False)
# stdf.plot_stops(max_users=1, map_f=m)

Device id:				51
Points of the original trajectory:	9295
Points of stops:			56
Device id:				52
Points of the original trajectory:	8437
Points of stops:			47
Device id:				54
Points of the original trajectory:	10923
Points of stops:			43
Device id:				53
Points of the original trajectory:	30857
Points of stops:			114
Device id:				58
Points of the original trajectory:	9103
Points of stops:			46
Device id:				59
Points of the original trajectory:	6701
Points of stops:			72
Device id:				60
Points of the original trajectory:	8556
Points of stops:			59
Device id:				61
Points of the original trajectory:	8747
Points of stops:			37
Device id:				62
Points of the original trajectory:	9364
Points of stops:			1
Device id:				63
Points of the original trajectory:	4405
Points of stops:			19
Device id:				64
Points of the original trajectory:	15047
Points of stops:			88
Device id:				65
Points of the original trajectory:	7429
Points of stops:			56
Device id:				79
Points of the original trajector